In [1]:
from xml2graph import build_graph_from_address
from profile2entities import profile_to_entities

profile = build_graph_from_address("part18.xml")  # или URL, или XML-строка
entities = profile_to_entities(profile, table_name_style="short")

# дальше entities — это dict, можно сериализовать/использовать в ETL


In [2]:
prompt = """

Следующий профиль в формате json задает собой реляционную структуру в БД. Твоя задача дать названия бизнес-сущностям и всем полям.

Профиль:

""" + str(entities) + """
Ответ выдай строго чистым JSON без комментариев ни до не после. Структура json ответа строго такая:

{
  "version": 1,
  "entities": [
    {
      "path": "<entity_path_from_entities_json>",
      "alias": "<entity_alias_snake_case_max63>",
      "title": "<entity_human_title>",
      "description": "<entity_description>",
      "fields": [
        {
          "path": "<field_rel_path_from_entities_json>",
          "alias": "<field_alias_snake_case_max63>",
          "title": "<field_human_title>",
          "description": "<field_description>"
        }
      ]
    },
    {
      "path": "<entity_path_from_entities_json_2>",
      "alias": "<entity2_alias_snake_case_max63>",
      "title": "<entity2_human_title>",
      "description": "<entity2_description>",
      "fields": [
        {
          "path": "<field_rel_path_from_entities_json_2>",
          "alias": "<field2_alias_snake_case_max63>",
          "title": "<field2_human_title>",
          "description": "<field2_description>"
        }
      ]
    }
  ]
}
"""

In [3]:
from entities_patch_validator import validate_patch
import json

with open("try1.json", "r", encoding="utf-8") as f:
    patch_json = json.load(f) 

# base_entities и patch_json — это уже загруженные dict'ы
result = validate_patch(entities, patch_json)
if result != "SUCCESS":
    # обработать ошибки
    print(result)

In [4]:
from final_profile import make_final_profile
final_spec = make_final_profile(entities, patch_json, "part18.xml")

In [5]:
final_spec

{'version': 1,
 'source': {'xml': 'part18.xml'},
 'typeset': 'canonical',
 'tables': [{'entity_path': '/root/item',
   'name': 'item',
   'alias': 'real_estate_object',
   'title': 'Объект недвижимости',
   'description': 'Основная сущность, представляющая объект недвижимости с кадастровыми, техническими и учетными характеристиками.',
   'table': 'real_estate_object',
   'columns': [{'name': 'id',
     'type': 'int64',
     'nullable': False,
     'role': 'pk_surrogate',
     'description': 'Суррогатный первичный ключ.'},
    {'name': 'unverified_purpose_code',
     'type': 'int64',
     'nullable': True,
     'source_path': 'actual_unverified_data/purpose/code',
     'is_key_hint': False,
     'observed': {'non_null': 4, 'nulls': 0, 'samples': 4},
     'title': 'Код назначения (неподтвержденные данные)',
     'description': 'Код предполагаемого назначения объекта, не прошедший проверку.'},
    {'name': 'unverified_purpose_name',
     'type': 'string',
     'nullable': True,
     'sour

In [6]:
final_spec = make_final_profile(entities, patch_json, r"C:\Users\1\CU\data_for_data_engineer\xml\part18.xml")

In [7]:
final_spec

{'version': 1,
 'source': {'xml': 'C:\\Users\\1\\CU\\data_for_data_engineer\\xml\\part18.xml'},
 'typeset': 'canonical',
 'tables': [{'entity_path': '/root/item',
   'name': 'item',
   'alias': 'real_estate_object',
   'title': 'Объект недвижимости',
   'description': 'Основная сущность, представляющая объект недвижимости с кадастровыми, техническими и учетными характеристиками.',
   'table': 'real_estate_object',
   'columns': [{'name': 'id',
     'type': 'int64',
     'nullable': False,
     'role': 'pk_surrogate',
     'description': 'Суррогатный первичный ключ.'},
    {'name': 'unverified_purpose_code',
     'type': 'int64',
     'nullable': True,
     'source_path': 'actual_unverified_data/purpose/code',
     'is_key_hint': False,
     'observed': {'non_null': 4, 'nulls': 0, 'samples': 4},
     'title': 'Код назначения (неподтвержденные данные)',
     'description': 'Код предполагаемого назначения объекта, не прошедший проверку.'},
    {'name': 'unverified_purpose_name',
     'typ

# DDL

In [8]:
# final_spec = make_final_profile(entities, patch_json, xml_path)

from ddl_postgres import generate_postgres_ddl
from ddl_clickhouse import generate_clickhouse_ddl

pg_sql = generate_postgres_ddl(final_spec, schema="public")
ch_sql = generate_clickhouse_ddl(final_spec, database="raw")

print(pg_sql)

CREATE SCHEMA IF NOT EXISTS public;
CREATE TABLE IF NOT EXISTS public.real_estate_object (
    id bigint NOT NULL,
    unverified_purpose_code bigint,
    unverified_purpose_name text,
    unverified_name text,
    unverified_area numeric(6,1),
    unverified_floors integer,
    unverified_underground_floors text,
    inventory_cost numeric(11,2),
    inventory_letter text,
    inventory_date text,
    inventory_name text,
    last_change_record_number text,
    last_container_fixed_at timestamptz NOT NULL,
    record_status text NOT NULL,
    cadastral_number text NOT NULL,
    was_previously_registered boolean,
    quarter_cadastral_number text NOT NULL,
    object_type_code bigint NOT NULL,
    object_type_name text NOT NULL,
    temporary_info text,
    object_definition text,
    formation_method_code integer,
    formation_method_name text,
    declared_area numeric(6,1),
    declared_floors text,
    wall_material_code bigint,
    wall_material_name text,
    declared_name text,

In [9]:
print(ch_sql)

CREATE DATABASE IF NOT EXISTS raw;

-- Объект недвижимости
-- Основная сущность, представляющая объект недвижимости с кадастровыми, техническими и учетными характеристиками.
-- UNIQUE (не применяется в CH): (cadastral_number, declared_purpose_code, record_number, section_number)  -- Наблюдаемый естественный ключ (из key_hints).
CREATE TABLE IF NOT EXISTS raw.real_estate_object (
    id Int64,
    unverified_purpose_code Nullable(Int64),
    unverified_purpose_name Nullable(String),
    unverified_name Nullable(String),
    unverified_area Nullable(Decimal(6,1)),
    unverified_floors Nullable(Int32),
    unverified_underground_floors Nullable(String),
    inventory_cost Nullable(Decimal(11,2)),
    inventory_letter Nullable(String),
    inventory_date Nullable(String),
    inventory_name Nullable(String),
    last_change_record_number Nullable(String),
    last_container_fixed_at DateTime('UTC'),
    record_status String,
    cadastral_number String,
    was_previously_registered Nulla

In [10]:
from ddlgenerator_postgres import generate_postgres_ddl
ddl_pg = generate_postgres_ddl(final_spec, schema="public", emit_unique=False)
print(ddl_pg)

# CH (как раньше; без реальных UNIQUE, только комментарии):
from ddlgenerator_clickhouse import generate_clickhouse_ddl
ddl_ch = generate_clickhouse_ddl(final_spec, database="raw", include_unique_comments=True)
print(ddl_ch)

CREATE SCHEMA IF NOT EXISTS public;
CREATE TABLE IF NOT EXISTS public.real_estate_object (
    id bigint NOT NULL,
    unverified_purpose_code bigint,
    unverified_purpose_name text,
    unverified_name text,
    unverified_area numeric(6,1),
    unverified_floors integer,
    unverified_underground_floors text,
    inventory_cost numeric(11,2),
    inventory_letter text,
    inventory_date text,
    inventory_name text,
    last_change_record_number text,
    last_container_fixed_at timestamptz NOT NULL,
    record_status text NOT NULL,
    cadastral_number text NOT NULL,
    was_previously_registered boolean,
    quarter_cadastral_number text NOT NULL,
    object_type_code bigint NOT NULL,
    object_type_name text NOT NULL,
    temporary_info text,
    object_definition text,
    formation_method_code integer,
    formation_method_name text,
    declared_area numeric(6,1),
    declared_floors text,
    wall_material_code bigint,
    wall_material_name text,
    declared_name text,

# DBML

In [12]:
from dbml_minimal import generate_dbml_minimal

# final_spec — твой объект профиля (dict)
dbml = generate_dbml_minimal(final_spec, with_project=False)
print(dbml)


Table real_estate_object {
  id bigint [not null, pk]
  unverified_purpose_code bigint
  unverified_purpose_name text
  unverified_name text
  unverified_area decimal(6,1)
  unverified_floors int
  unverified_underground_floors text
  inventory_cost decimal(11,2)
  inventory_letter text
  inventory_date text
  inventory_name text
  last_change_record_number text
  last_container_fixed_at timestamp [not null]
  record_status text [not null]
  cadastral_number text [not null]
  was_previously_registered boolean
  quarter_cadastral_number text [not null]
  object_type_code bigint [not null]
  object_type_name text [not null]
  temporary_info text
  object_definition text
  formation_method_code int
  formation_method_name text
  declared_area decimal(6,1)
  declared_floors text
  wall_material_code bigint
  wall_material_name text
  declared_name text
  permitted_use_name text
  declared_purpose_code bigint
  declared_purpose_name text
  year_built int
  year_commissioned text
  declared_

In [ ]:
print(dbml_str)

Project 'Registry ETL' {
  database_type: 'generic'
  note: 'Generated from final_spec'
}

Table real_estate_object {
  id bigint [not null, note: 'Суррогатный первичный ключ.', pk]
  unverified_purpose_code bigint [note: 'Код назначения (неподтвержденные данные)
Код предполагаемого назначения объекта, не прошедший проверку.']
  unverified_purpose_name text [note: 'Назначение (неподтвержденные данные)
Наименование предполагаемого назначения объекта, не прошедшее проверку.']
  unverified_name text [note: 'Наименование (неподтвержденные данные)
Наименование объекта, указанное без подтверждения.']
  unverified_area decimal(6,1) [note: 'Площадь (неподтвержденные данные)
Площадь объекта по неподтвержденным данным.']
  unverified_floors int [note: 'Этажность (неподтвержденные данные)
Количество этажей по неподтвержденным данным.']
  unverified_underground_floors text [note: 'Подземные этажи (неподтвержденные данные)
Количество подземных этажей по неподтвержденным данным.']
  inventory_cost d